# RESULTS

In [36]:
from pprint import pprint 
from DbConnector import DbConnector
from haversine import haversine, Unit
from tabulate import tabulate

In [37]:
try:
    connection = DbConnector()
    client = connection.client
    db = connection.db
except Exception as e:
    print("ERROR: Failed to connect to db:", e)

You are connected to the database: geolife
-----------------------------------------------



# PART 1

#### Showing the 10 first rows of each table in the database, after data is inserted

In [38]:
documents = db.user.find().limit(10)
for doc in documents: 
    pprint(doc)

{'_id': '000', 'has_labels': False}
{'_id': '001', 'has_labels': False}
{'_id': '002', 'has_labels': False}
{'_id': '003', 'has_labels': False}
{'_id': '004', 'has_labels': False}
{'_id': '005', 'has_labels': False}
{'_id': '006', 'has_labels': False}
{'_id': '007', 'has_labels': False}
{'_id': '008', 'has_labels': False}
{'_id': '009', 'has_labels': False}


In [39]:
documents = db.activity.find().limit(10)
for doc in documents:
    # Slice the trackpoints to only show the first 5 
    if 'trackpoints' in doc:
        doc['trackpoints'] = doc['trackpoints'][:5]
    pprint(doc)

{'_id': ObjectId('67151e95861279f7b5f1b143'),
 'end_time': datetime.datetime(2008, 10, 23, 11, 11, 12),
 'start_time': datetime.datetime(2008, 10, 23, 2, 53, 4),
 'trackpoints': [ObjectId('67151e95861279f7b5f1adb7'),
                 ObjectId('67151e95861279f7b5f1adb8'),
                 ObjectId('67151e95861279f7b5f1adb9'),
                 ObjectId('67151e95861279f7b5f1adba'),
                 ObjectId('67151e95861279f7b5f1adbb')],
 'transportation_mode': None,
 'user_id': '000'}
{'_id': ObjectId('67151e95861279f7b5f1b238'),
 'end_time': datetime.datetime(2008, 10, 24, 2, 47, 6),
 'start_time': datetime.datetime(2008, 10, 24, 2, 9, 59),
 'trackpoints': [ObjectId('67151e95861279f7b5f1b144'),
                 ObjectId('67151e95861279f7b5f1b145'),
                 ObjectId('67151e95861279f7b5f1b146'),
                 ObjectId('67151e95861279f7b5f1b147'),
                 ObjectId('67151e95861279f7b5f1b148')],
 'transportation_mode': None,
 'user_id': '000'}
{'_id': ObjectId('67151e9586

In [40]:
documents = db.trackpoint.find().limit(10)
for doc in documents:
    pprint(doc)

{'_id': ObjectId('67151e95861279f7b5f1adb7'),
 'altitude': '492',
 'date_days': '39744.1201851852',
 'date_time': datetime.datetime(2008, 10, 23, 2, 53, 4),
 'lat': '39.984702',
 'lon': '116.318417'}
{'_id': ObjectId('67151e95861279f7b5f1adb8'),
 'altitude': '492',
 'date_days': '39744.1202546296',
 'date_time': datetime.datetime(2008, 10, 23, 2, 53, 10),
 'lat': '39.984683',
 'lon': '116.31845'}
{'_id': ObjectId('67151e95861279f7b5f1adb9'),
 'altitude': '492',
 'date_days': '39744.1203125',
 'date_time': datetime.datetime(2008, 10, 23, 2, 53, 15),
 'lat': '39.984686',
 'lon': '116.318417'}
{'_id': ObjectId('67151e95861279f7b5f1adba'),
 'altitude': '492',
 'date_days': '39744.1203703704',
 'date_time': datetime.datetime(2008, 10, 23, 2, 53, 20),
 'lat': '39.984688',
 'lon': '116.318385'}
{'_id': ObjectId('67151e95861279f7b5f1adbb'),
 'altitude': '492',
 'date_days': '39744.1204282407',
 'date_time': datetime.datetime(2008, 10, 23, 2, 53, 25),
 'lat': '39.984655',
 'lon': '116.318263'}


# PART 2

#### 1. How many users, activities, and trackpoints are there in the dataset (after it is inserted into the database).

In [41]:
user_count = db.user.count_documents({})
activity_count = db.activity.count_documents({})
tp_count = db.trackpoint.count_documents({})
rows = [
    ["User", user_count],
    ["Activity", activity_count],
    ["Trackpoint", tp_count]
]
print(tabulate(rows, headers=["Collection", "Count"], tablefmt="pretty"))


+------------+---------+
| Collection |  Count  |
+------------+---------+
|    User    |   182   |
|  Activity  |  16050  |
| Trackpoint | 9686758 |
+------------+---------+


#### 2. Find the average number of activities per user.

In [51]:
pipeline = [
    {
        "$group": {
            "_id": "$user_id",
            "activity_count": {"$sum": 1}  # Count activities per user
        }
    },
    {
        "$group": {
            "_id": None,
            "average_activities_per_user": {"$avg": "$activity_count"}
        }
    }
]
result = db.activity.aggregate(pipeline)
for doc in result:
    pprint()

{'_id': None, 'average_activities_per_user': 92.77456647398844}


#### 3. Find the top 20 users with the highest number of activities.

In [53]:
pipeline = [
    {
        "$group": {
            "_id": "$user_id",
            "activity_count": {"$sum": 1}
        }
    },
    {
        "$sort": {
            'activity_count': -1
        }
    },
    {
        "$limit": 20
    }
]
result = db.activity.aggregate(pipeline)
for doc in result:
    pprint(doc)

{'_id': '128', 'activity_count': 2102}
{'_id': '153', 'activity_count': 1794}
{'_id': '025', 'activity_count': 715}
{'_id': '163', 'activity_count': 704}
{'_id': '062', 'activity_count': 691}
{'_id': '144', 'activity_count': 563}
{'_id': '041', 'activity_count': 399}
{'_id': '085', 'activity_count': 364}
{'_id': '004', 'activity_count': 346}
{'_id': '140', 'activity_count': 345}
{'_id': '167', 'activity_count': 320}
{'_id': '068', 'activity_count': 280}
{'_id': '017', 'activity_count': 265}
{'_id': '003', 'activity_count': 261}
{'_id': '014', 'activity_count': 236}
{'_id': '126', 'activity_count': 215}
{'_id': '030', 'activity_count': 210}
{'_id': '112', 'activity_count': 208}
{'_id': '011', 'activity_count': 201}
{'_id': '039', 'activity_count': 198}


#### 4. Find all users who have taken a taxi.

In [9]:
cursor.execute("""
    SELECT DISTINCT user_id
    FROM Activity
    WHERE transportation_mode = 'taxi'
""")
rows = cursor.fetchall()
print(tabulate(rows, headers=['User ID'], tablefmt="pretty"))


+---------+
| User ID |
+---------+
|   010   |
|   058   |
|   062   |
|   078   |
|   080   |
|   085   |
|   098   |
|   111   |
|   128   |
|   163   |
+---------+


#### 5. Find all types of transportation modes and count how many activities that are tagged with these transportation mode labels. Do not count the rows where the mode is null.

In [10]:
cursor.execute("""
    SELECT transportation_mode, COUNT(*) AS activity_count
    FROM Activity
    WHERE transportation_mode IS NOT NULL
    GROUP BY transportation_mode
    ORDER BY activity_count DESC
""")
rows = cursor.fetchall()
print(tabulate(rows, headers=['Transportation Mode', 'Activity Count'], tablefmt='pretty'))


+---------------------+----------------+
| Transportation Mode | Activity Count |
+---------------------+----------------+
|        walk         |      480       |
|         car         |      419       |
|        bike         |      263       |
|         bus         |      199       |
|       subway        |      133       |
|        taxi         |       37       |
|      airplane       |       3        |
|        train        |       2        |
|         run         |       1        |
|        boat         |       1        |
+---------------------+----------------+


#### 6. a) Find the year with the most activities.

In [11]:
cursor.execute("""
    SELECT YEAR(start_date_time) as year, COUNT(*) AS activity_count
    FROM Activity
    GROUP BY year
    ORDER BY activity_count DESC
""")
rows = cursor.fetchall()
print(tabulate(rows, headers=['Year', 'Activity Count'], tablefmt='pretty'))
print("\n", rows[0][0], "is the year with the most activities.")


+------+----------------+
| Year | Activity Count |
+------+----------------+
| 2008 |      5895      |
| 2009 |      5880      |
| 2010 |      1488      |
| 2011 |      1204      |
| 2007 |      994       |
| 2012 |      588       |
| 2000 |       1        |
+------+----------------+

 2008 is the year with the most activities.


#### 6. b) Is this also the year with most recorded hours? 

In [12]:
cursor.execute("""
    SELECT YEAR(start_date_time) as year, 
        ROUND(SUM(TIMESTAMPDIFF(SECOND, start_date_time, end_date_time))/3600, 0) as total_hours
    FROM Activity
    GROUP BY year
    ORDER BY total_hours DESC
""")
rows = cursor.fetchall()
print(tabulate(rows, headers=['Year', 'Total Hours'], tablefmt='pretty'))
print("\n", rows[0][0], "is the year with the most hours.")

+------+-------------+
| Year | Total Hours |
+------+-------------+
| 2009 |    11616    |
| 2008 |    9201     |
| 2007 |    2315     |
| 2010 |    1395     |
| 2011 |    1132     |
| 2012 |     711     |
| 2000 |      0      |
+------+-------------+

 2009 is the year with the most hours.


#### 7. Find the total distance (in km) walked in 2008, by user with id=112.

In [13]:
cursor.execute("""
    SELECT tp.lat, tp.lon
    FROM TrackPoint tp
    JOIN Activity a ON tp.activity_id = a.id
    WHERE a.user_id = '112'
      AND a.transportation_mode = 'walk'
      AND YEAR(a.start_date_time) = 2008
    ORDER BY a.id, tp.date_time
""")
trackpoints = cursor.fetchall()

total_distance_km = 0
for i in range(1, len(trackpoints)):
    prev_point = (trackpoints[i-1][0], trackpoints[i-1][1])  # (lat, lon)
    curr_point = (trackpoints[i][0], trackpoints[i][1])      # (lat, lon)
    distance = haversine(prev_point, curr_point, unit=Unit.KILOMETERS)
    total_distance_km += distance

print(f"Total distance walked in 2008 by user 112: {total_distance_km:.2f} km")

Total distance walked in 2008 by user 112: 141.21 km


#### 8. Find the top 20 users who have gained the most altitude meters.

In [14]:
cursor.execute("""
SELECT
    a.user_id AS user_id,
    SUM(CASE WHEN tp2.altitude > tp1.altitude THEN tp2.altitude - tp1.altitude ELSE 0 END) 
    AS total_elevation_gain
FROM Activity AS a 
JOIN TrackPoint AS tp1 ON a.id = tp1.activity_id
JOIN TrackPoint AS tp2 ON a.id = tp2.activity_id AND tp2.id = tp1.id + 1
WHERE tp1.altitude IS NOT NULL AND tp2.altitude IS NOT NULL 
    AND tp1.altitude != -777 AND tp2.altitude != -777
GROUP BY user_id
ORDER BY total_elevation_gain DESC
LIMIT 20
""")

rows = cursor.fetchall()
print(tabulate(rows, headers=['User ID', 'Total Elevation Gain (feet)'], tablefmt='pretty'))
print("""\n I only remove altitude values which we KNOW are invalid, 
      ie. they are -777 as specified it the assignment or NULL.""")

+---------+-----------------------------+
| User ID | Total Elevation Gain (feet) |
+---------+-----------------------------+
|   128   |           2135455           |
|   153   |           1823683           |
|   004   |           1089358           |
|   041   |           789890            |
|   003   |           766613            |
|   085   |           714049            |
|   163   |           673439            |
|   062   |           596103            |
|   144   |           588771            |
|   030   |           576377            |
|   039   |           481311            |
|   084   |           430319            |
|   000   |           398638            |
|   002   |           377503            |
|   167   |           370647            |
|   025   |           358098            |
|   037   |           325528            |
|   140   |           311151            |
|   126   |           272389            |
|   017   |           205270            |
+---------+-----------------------

#### 9. Find all users who have invalid activities, and the number of invalid activities per user

In [15]:
cursor.execute("""
    WITH InvalidActivities AS (
        SELECT
            tp1.activity_id
        FROM
            TrackPoint tp1
        JOIN
            TrackPoint tp2 ON tp1.activity_id = tp2.activity_id
                        AND tp2.id = tp1.id + 1
        WHERE
            TIMESTAMPDIFF(MINUTE, tp1.date_time, tp2.date_time) >= 5
        GROUP BY
            tp1.activity_id
    )
    SELECT  
        user_id,
        COUNT(activity_id) AS invalid_activities_count
    FROM
        InvalidActivities as ia
    JOIN Activity as a ON a.id = ia.activity_id
    GROUP BY
        user_id
""")
rows = cursor.fetchall()
print(tabulate(rows, headers=['User ID', 'Invalid Activity Count'], tablefmt='pretty'))

+---------+------------------------+
| User ID | Invalid Activity Count |
+---------+------------------------+
|   000   |          101           |
|   001   |           45           |
|   002   |           98           |
|   003   |          179           |
|   004   |          219           |
|   005   |           45           |
|   006   |           17           |
|   007   |           30           |
|   008   |           16           |
|   009   |           31           |
|   010   |           50           |
|   011   |           32           |
|   012   |           43           |
|   013   |           29           |
|   014   |          118           |
|   015   |           46           |
|   016   |           20           |
|   017   |          129           |
|   018   |           27           |
|   019   |           31           |
|   020   |           20           |
|   021   |           7            |
|   022   |           55           |
|   023   |           12           |
|

#### 10. Find the users who have tracked an activity in the Forbidden City of Beijing.  


In [16]:
cursor.execute("""
    SELECT user_id
    FROM Activity a
    JOIN TrackPoint tp ON a.id = tp.activity_id
    WHERE tp.lat BETWEEN 39.915 AND 39.917
      AND tp.lon BETWEEN 116.396 AND 116.398
    GROUP BY user_id
""")
rows = cursor.fetchall()
print(tabulate(rows, headers=['User ID'], tablefmt='pretty'))
print("\n I provided a tiny range around the given coordinates to simplify the query.")

+---------+
| User ID |
+---------+
|   004   |
|   018   |
|   019   |
|   131   |
+---------+

 I provided a tiny range around the given coordinates to simplify the query.


#### 11. Find all users who have registered transportation_mode and their most used transportation_mode. 

In [17]:

cursor.execute("""
    SELECT user_id, transportation_mode 
    FROM
        (SELECT user_id, transportation_mode, 
        RANK() OVER(PARTITION BY user_id ORDER BY transportation_mode ASC) as lexrnk 
        FROM
            (SELECT user_id, transportation_mode, COUNT(*) AS activity_count,
            RANK() OVER(PARTITION BY user_id ORDER BY COUNT(*) DESC) as rnk
            FROM Activity
            WHERE transportation_mode IS NOT NULL
            GROUP BY user_id, transportation_mode) AS Ranked
        WHERE rnk = 1
        ) AS LexRanked
    WHERE lexrnk = 1
    ORDER BY user_id
    """)
rows = cursor.fetchall()
print(tabulate(rows, headers=['User ID',' Most Used Transportation Mode'], tablefmt='pretty'))
print("""\n The innermost subquery finds the user's most used transportation mode(s). 
      The second subquery handles tie breaks alphabetically.""")

+---------+--------------------------------+
| User ID |  Most Used Transportation Mode |
+---------+--------------------------------+
|   010   |              taxi              |
|   020   |              bike              |
|   021   |              walk              |
|   052   |              bus               |
|   056   |              bike              |
|   058   |              car               |
|   060   |              walk              |
|   062   |              bus               |
|   064   |              bike              |
|   065   |              bike              |
|   067   |              walk              |
|   069   |              bike              |
|   073   |              walk              |
|   075   |              walk              |
|   076   |              car               |
|   078   |              walk              |
|   080   |              bike              |
|   081   |              bike              |
|   082   |              walk              |
|   084   